In [29]:
import cv2
import numpy as np
from rocket import Rocket

task = 'hover'  # 'hover' or 'landing'
max_episode = 5
max_steps = 800


import random
import torch
from collections import deque

alpha = 0.001
gamma = 0.99
lmbda         = 0.99
eps_clip      = 0.1
K_epoch       = 4

env = Rocket(task=task, max_steps=max_steps)


class VNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fcV1 = torch.nn.Linear(8, 256)
        self.fcV2 = torch.nn.Linear(256, 256)
        #add an additional layer
#         self.fcV3 = torch.nn.Linear(256, 512)
        self.fcV3 = torch.nn.Linear(256, 1)
        
    def forward(self, x):
        x = self.fcV1(x)
        x = torch.nn.functional.relu(x)
        x = self.fcV2(x)
        x = torch.nn.functional.relu(x)
        x = self.fcV3(x)
#         x = torch.nn.functional.relu(x)
#         x = self.fcV4(x)
        return x
    
class PolicyNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fcA1 = torch.nn.Linear(8, 256)
        self.fcA2 = torch.nn.Linear(256, 256)
        #add one layer
#         self.fcA3 = torch.nn.Linear(256, 512)
        self.fcA3_thrust = torch.nn.Linear(256,3)
        self.fcA3 = torch.nn.Linear(256, 9)
        
    def forward(self, x):
        x = self.fcA1(x)
        x = torch.nn.functional.relu(x)
        x = self.fcA2(x)
        x = torch.nn.functional.relu(x)
        x = self.fcA3(x)  
#         x = torch.nn.functional.relu(x)
#         x = self.fcA4(x)  
        x = torch.nn.functional.softmax(x, dim=-1)
        
        return x

In [4]:
import os
os.getcwd()

'/Users/joonpark/Desktop/KAIST_Y4S2/강화학습개론/FinalProject/rocket-recycling'

In [30]:
#load saved networks for testing:
base_path = os.getcwd() + '/checkpoints'
date = '2023-05-20'

pi_test = PolicyNetwork()
# V_test = VNetwork()

# v_path = base_path + '/VNetwork/' + date + '_best_v_state.pt'
pi_path = base_path + '/PolicyNetwork/' + date + '_best_pi_state.pt'
# V_test.load_state_dict(torch.load(v_path))
pi_test.load_state_dict(torch.load(pi_path))

# pi_test.cuda()
pi_test.eval()

# V_test.cuda()
# V_test.eval()

PolicyNetwork(
  (fcA1): Linear(in_features=8, out_features=256, bias=True)
  (fcA2): Linear(in_features=256, out_features=256, bias=True)
  (fcA3_thrust): Linear(in_features=256, out_features=3, bias=True)
  (fcA3): Linear(in_features=256, out_features=9, bias=True)
)

In [21]:
print(pi_test)

PolicyNetwork(
  (fcA1): Linear(in_features=8, out_features=256, bias=True)
  (fcA2): Linear(in_features=256, out_features=256, bias=True)
  (fcA3): Linear(in_features=256, out_features=512, bias=True)
  (fcA4): Linear(in_features=512, out_features=9, bias=True)
)


In [31]:


for episode in range(max_episode):
    state = env.reset()
    
    score = 0
    rewards= []
    
    for step in range(max_steps):
        probs_target = pi_test(torch.FloatTensor(state))
        action = torch.multinomial(probs_target, 1).item()
        state, reward, done, _ = env.step(action)
        env.render()
        
        score += reward
        
        if done or step == max_steps-1:
            rewards.append(score)
            cv2.destroyAllWindows()
            break

average = np.mean(np.array(rewards))
print(f'average rewards = {average}')


average rewards = 69.7398674038349
